In [2]:
import pandas as pd
import os
from functools import reduce

# --- 1. Set paths ---
data_folder = 'health-financing/data'          # Folder with 12 indicator CSVs
codes_folder = 'health-financing/codes'        # Folder with COUNTRY.csv, REGION.csv, etc.

# --- 2. Load COUNTRY.csv and inspect structure ---
country_file = os.path.join(codes_folder, 'COUNTRY.csv')
country_df = pd.read_csv(country_file)

# DEBUG: Show structure
print("COUNTRY.csv Columns:", country_df.columns.tolist())
print("Sample:")
print(country_df.head())

# Use actual column names from your data
country_df = country_df[['Code', 'Title', 'ParentCode', 'ParentTitle']].rename(
    columns={
        'Code': 'SpatialDimensionValueCode',
        'Title': 'CountryName',
        'ParentCode': 'RegionCode',
        'ParentTitle': 'RegionName'
    }
)

# Filter: Keep only African countries
african_countries = country_df[country_df['RegionName'] == 'Africa'].copy()
african_codes = african_countries['SpatialDimensionValueCode'].tolist()

print(f"✅ Found {len(african_codes)} African countries.")

# --- 3. Define indicator mapping: file name → desired column name ---
indicator_mapping = {
    'GHED_CHEGDP_SHA2011': 'CHE_GDP',                    # Health expenditure % GDP
    'GHED_CHE_pc_US_SHA2011': 'CHE_per_capita',          # Health spending per capita (USD)
    'GHED_GGHE-DCHE_SHA2011': 'GGHE_pct_CHE',            # Govt share of health spending
    'GHED_GGHE-DGGE_SHA2011': 'GGHE_pct_GGE',            # Govt health spending % total govt spending
    'GHED_GGHE-DGDP_SHA2011': 'GGHE_pct_GDP',            # Govt health spending % GDP
    'GHED_GGHE-D_pc_US_SHA2011': 'GGHE_per_capita',      # Govt health spending per capita (USD)
    'GHED_PVT-DCHE_SHA2011': 'PVT_pct_CHE',              # Private share of health spending
    'GHED_PVT-D_pc_US_SHA2011': 'PVT_per_capita',        # Private spending per capita (USD)
    'GHED_EXTCHE_SHA2011': 'EXT_pct_CHE',                # External (aid) share of health spending
    'GHED_EXT_pc_US_SHA2011': 'EXT_per_capita',          # External spending per capita (USD)
    'GHED_OOP_pc_US_SHA2011': 'OOP_per_capita',          # Out-of-pocket per capita (USD)
    'GHED_OOPSCHE_SHA2011': 'OOP_pct_CHE',               # Out-of-pocket % of health spending
}

# --- 4. Load and clean all 12 indicator files ---
all_dfs = []

for filename, col_name in indicator_mapping.items():
    file_path = os.path.join(data_folder, f'{filename}.csv')
    
    if not os.path.exists(file_path):
        print(f"⚠️ Warning: {file_path} not found!")
        continue

    # Load data
    df = pd.read_csv(file_path)
    
    # DEBUG: Check columns
    if 'SpatialDimensionValueCode' not in df.columns:
        print(f"❌ Error: 'SpatialDimensionValueCode' not in {filename}")
        continue

    # Extract relevant columns
    df = df[['SpatialDimensionValueCode', 'TimeDim', 'NumericValue']].copy()
    df = df.rename(columns={'TimeDim': 'Year', 'NumericValue': col_name})
    
    # Convert to numeric
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
    
    # Keep only African countries
    df = df[df['SpatialDimensionValueCode'].isin(african_codes)]
    
    all_dfs.append(df)

# --- 5. Merge all indicators ---
if not all_dfs:
    raise Exception("❌ No indicator data loaded. Check file paths and column names.")

# Sort by size (largest first) for better merge stability
all_dfs.sort(key=len, reverse=True)

# Merge all DataFrames
combined_df = reduce(
    lambda left, right: pd.merge(left, right, on=['SpatialDimensionValueCode', 'Year'], how='outer'),
    all_dfs
)

# --- 6. Add country and region names ---
combined_df = combined_df.merge(
    african_countries[['SpatialDimensionValueCode', 'CountryName', 'RegionName']],
    on='SpatialDimensionValueCode',
    how='left'
)

# --- 7. Filter years: 2000–2023 ---
combined_df = combined_df[(combined_df['Year'] >= 2000) & (combined_df['Year'] <= 2023)]

# --- 8. Sort and clean ---
combined_df = combined_df.sort_values(['CountryName', 'Year']).reset_index(drop=True)

# Reorder columns
col_order = ['CountryName', 'SpatialDimensionValueCode', 'Year', 'RegionName'] + list(indicator_mapping.values())
combined_df = combined_df[col_order]

# --- 9. Save unified dataset ---
output_file = 'unified_african_health_financing.csv'
combined_df.to_csv(output_file, index=False)
print(f"✅ Unified dataset saved to {output_file}")
print(f"📊 Shape: {combined_df.shape} (rows, columns)")
print(f"📅 Years: {combined_df['Year'].min()} to {combined_df['Year'].max()}")
print(f"🌍 Countries: {combined_df['CountryName'].nunique()}")

COUNTRY.csv Columns: ['Dimension', 'Code', 'Title', 'ParentDimension', 'ParentCode', 'ParentTitle']
Sample:
  Dimension Code        Title ParentDimension ParentCode  \
0   COUNTRY  ABW        Aruba          REGION        AMR   
1   COUNTRY  AFG  Afghanistan          REGION        EMR   
2   COUNTRY  AGO       Angola          REGION        AFR   
3   COUNTRY  AIA     Anguilla          REGION        AMR   
4   COUNTRY  ALB      Albania          REGION        EUR   

             ParentTitle  
0               Americas  
1  Eastern Mediterranean  
2                 Africa  
3               Americas  
4                 Europe  
✅ Found 50 African countries.
✅ Unified dataset saved to unified_african_health_financing.csv
📊 Shape: (1054, 16) (rows, columns)
📅 Years: 2000 to 2022
🌍 Countries: 47
